##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Aprendizaje por transferencia con TensorFlow Hub

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/images/transfer_learning_with_hub.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/images/transfer_learning_with_hub.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver en GitHub</a> </td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/images/transfer_learning_with_hub.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
  <td>     <a href="https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo en TF Hub</a>
</td>
</table>

[TensorFlow Hub](https://tfhub.dev/) es un repositorio de modelos de TensorFlow preentrenados.

En este tutorial se muestra cómo:

1. Utilizar modelos de TensorFlow Hub con `tf.keras`.
2. Utilizar un modelo de clasificación de imágenes de TensorFlow Hub.
3. Realizar un aprendizaje por transferencia simple para afinar un modelo para sus propias clases de imágenes.

## Preparación

In [ ]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime

%load_ext tensorboard

## Un clasificador de ImageNet

Comenzará utilizando un modelo clasificador preentrenado en el conjunto de datos de referencia [ImageNet](https://en.wikipedia.org/wiki/ImageNet): ¡no se requiere entrenamiento inicial!

### Descargue el clasificador

Seleccione un modelo <a href="https://arxiv.org/abs/1801.04381" class="external">MobileNetV2</a> preentrenado [de TensorFlow Hub](https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2) y envuélvalo como una capa de Keras con [`hub.KerasLayer`](https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer). Cualquier <a href="https://tfhub.dev/s?q=tf2&amp;module-type=image-classification/" class="external">modelo de clasificador de imágenes compatible</a> de TensorFlow Hub funcionará aquí, incluyendo los ejemplos proporcionados en el menú desplegable que aparece a continuación.

In [ ]:
mobilenet_v2 ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
inception_v3 = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"

classifier_model = mobilenet_v2 #@param ["mobilenet_v2", "inception_v3"] {type:"raw"}

In [ ]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,))
])

### Ejecútelo en una sola imagen

Descargue una sola imagen para probar el modelo:

In [ ]:
grace_hopper = tf.keras.utils.get_file('image.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
grace_hopper = Image.open(grace_hopper).resize(IMAGE_SHAPE)
grace_hopper

In [ ]:
grace_hopper = np.array(grace_hopper)/255.0
grace_hopper.shape

Agregue una dimensión de lote (con `np.newaxis`) y pase la imagen al modelo:

In [ ]:
result = classifier.predict(grace_hopper[np.newaxis, ...])
result.shape

El resultado es un vector de logits de 1001 elementos, que califica la probabilidad de cada clase en relación con la imagen.

El ID de la clase superior se puede encontrar con `tf.math.argmax`:

In [ ]:
predicted_class = tf.math.argmax(result[0], axis=-1)
predicted_class

### Descodifique las predicciones

Tome el ID de `predicted_class` (como `653`) y obtenga las etiquetas del conjunto de datos de ImageNet para decodificar las predicciones:

In [ ]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [ ]:
plt.imshow(grace_hopper)
plt.axis('off')
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title("Prediction: " + predicted_class_name.title())

## Aprendizaje por transferencia simple

Pero, ¿qué ocurre si desea crear un clasificador personalizado utilizando su propio conjunto de datos que tiene clases que no están incluidas en el conjunto de datos de ImageNet original (con el que se entrenó el modelo preentrenado)?

Para hacer eso, puede:

1. Seleccione un modelo preentrenado de TensorFlow Hub; y
2. Vuelva a entrenar la capa superior (la última) para que reconozca las clases de su conjunto de datos personalizado.

### Conjunto de datos

En este ejemplo, utilizará el conjunto de datos de flores en TensorFlow:

In [ ]:
import pathlib

data_file = tf.keras.utils.get_file(
  'flower_photos.tgz',
  'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
  cache_dir='.',
   extract=True)

data_root = pathlib.Path(data_file).with_suffix('')

En primer lugar, cargue estos datos en el modelo utilizando los datos de imagen del disco con `tf.keras.utils.image_dataset_from_directory`, lo que generará un `tf.data.Dataset`:

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

El conjunto de datos de flores tiene cinco clases:

In [ ]:
class_names = np.array(train_ds.class_names)
print(class_names)

En segundo lugar, dado que la convención de TensorFlow Hub para los modelos de imagen es esperar entradas flotantes en el rango `[0, 1]`, utilice la capa de preprocesamiento `tf.keras.layers.Rescaling` para conseguir esto.

Nota: También podría incluir la capa `tf.keras.layers.Rescaling` dentro del modelo. Consulte la guía [Trabajar con capas de preprocesamiento](https://www.tensorflow.org/guide/keras/preprocessing_layers) para conocer las ventajas y desventajas.

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

Por último, finalice la canalización de entrada utilizando la precarga en el búfer con `Dataset.prefetch`, de modo que pueda obtener los datos del disco sin problemas de bloqueo de E/S.

Estos son algunos de los métodos `tf.data` más importantes que debe utilizar durante la carga de datos. Los lectores interesados pueden obtener más información sobre ellos, así como sobre cómo almacenar datos en el caché del disco y otras técnicas, en la guía [Mejor rendimiento con API tf.data](https://www.tensorflow.org/guide/data_performance#prefetching).

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

### Ejecute el clasificador en un lote de imágenes

Ahora, ejecute el clasificador en un lote de imágenes:

In [ ]:
result_batch = classifier.predict(train_ds)

In [ ]:
predicted_class_names = imagenet_labels[tf.math.argmax(result_batch, axis=-1)]
predicted_class_names

Revise cómo se alinean estas predicciones con las imágenes:

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')
_ = plt.suptitle("ImageNet predictions")

Nota: todas las imágenes son licencia de CC-BY, los creadores están en el archivo LICENSE.txt.

Los resultados distan mucho de ser perfectos, pero son razonables si se tiene en cuenta que no son las clases para las que se entrenó el modelo (excepto para "daisy" (margarita)).

### Descargue el modelo sin la capa superior

TensorFlow Hub también distribuye modelos sin la capa superior de clasificación. Estos pueden utilizarse para realizar fácilmente el aprendizaje por transferencia.

Seleccione un modelo <a href="https://arxiv.org/abs/1801.04381" class="external">MobileNetV2</a> preentrenado <a href="https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4" class="external">de TensorFlow Hub</a>. Cualquier modelo <a href="https://tfhub.dev/s?module-type=image-feature-vector&amp;q=tf2" class="external">compatible con vectores de características de imagen</a> de TensorFlow Hub funcionará aquí, incluyendo los ejemplos del menú desplegable.

In [ ]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"

feature_extractor_model = mobilenet_v2 #@param ["mobilenet_v2", "inception_v3"] {type:"raw"}

Cree el extractor de características envolviendo el modelo preentrenado como una capa de Keras con [`hub.KerasLayer`](https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer). Utilice el argumento `trainable=False` para congelar las variables, de forma que el entrenamiento sólo modifique la nueva capa clasificadora:

In [ ]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

El extractor de características devuelve un vector de 1280 longitudes para cada imagen (el tamaño del lote de imágenes se mantiene en 32 en este ejemplo):

In [ ]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

### Coloque una capa superior de clasificación

Para completar el modelo, envuelva la capa del extractor de características en un modelo `tf.keras.Sequential` y agregue una capa totalmente conectada para realizar la clasificación:

In [ ]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()

In [ ]:
predictions = model(image_batch)

In [ ]:
predictions.shape

### Entrene el modelo

Utilice `Model.compile` para configurar el proceso de entrenamiento y agregue una retrollamada `tf.keras.callbacks.TensorBoard` para crear y almacenar registros:

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.

Ahora utilice el método `Model.fit` para entrenar el modelo.

Para mantener este ejemplo corto, se entrenará durante sólo 10 épocas. Para visualizar el progreso del entrenamiento en TensorBoard más tarde, cree y almacene los registros en una [retrollamada a TensorBoard](https://www.tensorflow.org/tensorboard/get_started#using_tensorboard_with_keras_modelfit).

In [ ]:
NUM_EPOCHS = 10

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

Inicie el TensorBoard para ver cómo cambian las métricas con cada época y para realizar un seguimiento de otros valores escalares:

In [ ]:
%tensorboard --logdir logs/fit

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/images/tensorboard_transfer_learning_with_hub.png?raw=1"/> -->

### Verifique las predicciones

Obtenga la lista ordenada de nombres de clases a partir de las predicciones del modelo:

In [ ]:
predicted_batch = model.predict(image_batch)
predicted_id = tf.math.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]
print(predicted_label_batch)

Represente las predicciones del modelo:

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)

for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_label_batch[n].title())
  plt.axis('off')
_ = plt.suptitle("Model predictions")

## Exporte y vuelva a cargar su modelo

Ahora que ya entrenó el modelo, expórtelo como SavedModel para volver a utilizarlo más adelante.

In [ ]:
t = time.time()

export_path = "/tmp/saved_models/{}".format(int(t))
model.save(export_path)

export_path

Confirme que puede volver a cargar SavedModel y que el modelo es capaz de producir los mismos resultados:

In [ ]:
reloaded = tf.keras.models.load_model(export_path)

In [ ]:
result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

In [ ]:
abs(reloaded_result_batch - result_batch).max()

In [ ]:
reloaded_predicted_id = tf.math.argmax(reloaded_result_batch, axis=-1)
reloaded_predicted_label_batch = class_names[reloaded_predicted_id]
print(reloaded_predicted_label_batch)

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(reloaded_predicted_label_batch[n].title())
  plt.axis('off')
_ = plt.suptitle("Model predictions")

## Siguientes pasos

Puede utilizar SavedModel para cargarlo para realizar inferencias o convertirlo en un modelo [TensorFlow Lite](https://www.tensorflow.org/lite/models/convert/) (para aprendizaje automático en el dispositivo) o en un modelo [TensorFlow.js](https://www.tensorflow.org/js/tutorials#convert_pretrained_models_to_tensorflowjs) (para que realice un aprendizaje automático en JavaScript).

Descubra [más tutoriales](https://www.tensorflow.org/hub/tutorials) para aprender a utilizar modelos preentrenados de TensorFlow Hub en tareas de imagen, texto, audio y video.